In [1]:
import torch
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.insert(1, '../')

from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import re

from ssnmf import SSNMF

## Loading and Preprocessing 20 Newsgroup Dataset

In [2]:
remove = ('headers','footers','quotes')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['thanks','edu','also','would','one','could','please','really','many','anyone','good','right','get','even','want','must','something','well','much','still','said','stay','away','first','looking','things','try','take','look','make','may','include','thing','like','two','or','etc','phone','oh','email'])


categories = [
 'comp.graphics',

 'comp.sys.mac.hardware',
 'misc.forsale',
 'rec.motorcycles',
 'rec.sport.baseball',
 'sci.med',
 'sci.space',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.religion.misc'
 ]

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)

# remove numbers
data_cleaned = [re.sub(r'\d+','', file) for file in newsgroups_train.data]

vectorizer = TfidfVectorizer(stop_words=stopwords_list)
vectors = vectorizer.fit_transform(data_cleaned).transpose()
idx_to_word = np.array(vectorizer.get_feature_names())

X = vectors
d, n = np.shape(X)
Y = np.zeros((n))

labels = {0:0, 1:0, 2:1, 3:2, 4:2, 5:3, 6:3, 7:4, 8:4, 9:5}

for i in range(n-1):
    label = newsgroups_train.target[i]
    Y[i] = label

X = torch.from_numpy(X.todense())
Y = torch.from_numpy(Y).long()


m = X.shape[0]
k1 = 10
k2 = 6

sub = 100 #HOW MANY PER CLASS
count = np.zeros((k1))

X_new = torch.zeros((X.shape[0], sub*k1))
Y_new = torch.zeros((sub*k1)).long()
Y_10 = torch.zeros((sub*k1)).long()
j = 0
for i in range(Y.shape[0]):
    if(count[Y[i]] >= sub):
        continue
    count[Y[i]] += 1
    X_new[:,j] = X[:,i]
    Y_new[j] = labels[int(Y[i])]
    Y_10[j] = int(Y[i])
    j += 1

X = X_new
Y = Y_new

ind = np.argsort(Y_10)
X = X[:,ind]
Y = Y[ind]
Y_10 = Y_10[ind]

split = 0.75
L = np.zeros((6, Y.shape[0]))
for i in range(10):
    L[:,i*sub:i*sub+(int(split*sub))] = 1

X = np.asarray(X)
Y = np.asarray(Y)

In [3]:
Y_hot = np.zeros((Y.shape[0], Y.max()+1))
Y_hot[np.arange(Y.shape[0]),Y] = 1
Y_hot = Y_hot.T

### Run First layer of Unsupervised NMF

In [4]:
results_all_layer_1 = []
results_all_layer_2_unsup = []
results_all_layer_2_sup = []

for i in range(0,10):
    
    
    # Run First Layer of Unsupervised NMF
    # ------------------------------------
    
    np.random.seed(i)
    model_1 = SSNMF(X,10,modelNum=1)
    N = 800
    model_1.mult(numiters = N)

    results_layer_1 = {}
    results_layer_1['A'] = model_1.A
    results_layer_1['S'] = model_1.S
    results_all_layer_1.append(results_layer_1)
    
    
    # Run Second Layer of Unsupervised NMF
    # ------------------------------------
    
    np.random.seed(i)
    model_2 = SSNMF(model_1.S,6,modelNum=1)
    N = 800
    model_2.mult(numiters = N)
    
    results_layer_2_unsup = {}
    results_layer_2_unsup['A'] = model_2.A
    results_layer_2_unsup['S'] = model_2.S
    results_all_layer_2_unsup.append(results_layer_2_unsup)
    
    
    # Run Second Layer of Supervised NMF
    # ------------------------------------
    
    np.random.seed(i)
    model_3 = SSNMF(model_1.S,6,Y = Y_hot,L=L,lam=15, modelNum=3)
    N = 800
    model_3.mult(numiters = N)
    
    results_layer_2_sup = {}
    results_layer_2_sup['A'] = model_3.A
    results_layer_2_sup['S'] = model_3.S
    results_all_layer_2_sup.append(results_layer_2_sup)
    
    print("Done with run...   ", i+1)

Done with run...    1
Done with run...    2
Done with run...    3
Done with run...    4
Done with run...    5
Done with run...    6
Done with run...    7
Done with run...    8
Done with run...    9
Done with run...    10


In [5]:
acc_all_layer_1 = []
acc_all_layer_2_unsup = []
acc_all_layer_2_sup = []


for i in range(10):
    
    results_layer_1 = results_all_layer_1[i]
    results_layer_2_unsup = results_all_layer_2_unsup[i]
    results_layer_2_sup = results_all_layer_2_sup[i]
    
    B = np.multiply(Y_hot,L) @ np.linalg.pinv(results_layer_1['S'])
    Y_pred = np.argmax(np.dot(B,results_layer_1['S']), axis=0)
    acc = Y[L[0]==0][Y_pred[L[0]==0]==Y[L[0]==0]].shape[0] / Y[L[0]==0].shape[0]
    acc_all_layer_1.append(acc)
    
    B = np.multiply(Y_hot,L) @ np.linalg.pinv(results_layer_2_unsup['S'])
    Y_pred = np.argmax(np.dot(B,results_layer_2_unsup['S']), axis=0)
    acc = Y[L[0]==0][Y_pred[L[0]==0]==Y[L[0]==0]].shape[0] / Y[L[0]==0].shape[0]
    acc_all_layer_2_unsup.append(acc)
    
    B = np.multiply(Y_hot,L) @ np.linalg.pinv(results_layer_2_sup['S'])
    Y_pred = np.argmax(np.dot(B,results_layer_2_sup['S']), axis=0)
    acc = Y[L[0]==0][Y_pred[L[0]==0]==Y[L[0]==0]].shape[0] / Y[L[0]==0].shape[0]
    acc_all_layer_2_sup.append(acc)

In [6]:
print("Layer 1 unsupervised accuracy...  ", sum(acc_all_layer_1) / 10)
print("Layer 2 unsupervised accuracy...  ", sum(acc_all_layer_2_unsup) / 10)
print("Layer 2 supervised accuracy...  ", sum(acc_all_layer_2_sup) / 10)

Layer 1 unsupervised accuracy...   0.5932000000000001
Layer 2 unsupervised accuracy...   0.5071999999999999
Layer 2 supervised accuracy...   0.546


In [ ]:
A_0 = model_1.A
S_1 = model_1.S

B = np.multiply(Y_hot,L) @ np.linalg.pinv(S_1)

Y_pred = np.argmax(np.dot(B,S_1), axis=0)

#Y = Y.numpy()
print("Accuracy: {}/{}".format(Y[Y_pred==Y].shape[0], Y.shape[0]))
print("Accuracy: {}/{}".format(Y[L[0]==1][Y_pred[L[0]==1]==Y[L[0]==1]].shape[0], Y[L[0]==1].shape[0]))
print("Accuracy: {}/{}".format(Y[L[0]==0][Y_pred[L[0]==0]==Y[L[0]==0]].shape[0], Y[L[0]==0].shape[0]))

### Run Second Layer of Unsupervised NMF

In [ ]:
np.random.seed(1)

model_2 = SSNMF(S_1,6,modelNum=1)

N=3000
model_2.mult(numiters = N)

In [ ]:
A_1 = model_2.A
S_2 = model_2.S

B = np.multiply(Y_hot,L) @ np.linalg.pinv(S_2)

Y_pred = np.argmax(np.dot(B,S_2), axis=0)

#Y = Y.numpy()
print("Accuracy: {}/{}".format(Y[Y_pred==Y].shape[0], Y.shape[0]))
print("Accuracy: {}/{}".format(Y[L[0]==1][Y_pred[L[0]==1]==Y[L[0]==1]].shape[0], Y[L[0]==1].shape[0]))
print("Accuracy: {}/{}".format(Y[L[0]==0][Y_pred[L[0]==0]==Y[L[0]==0]].shape[0], Y[L[0]==0].shape[0]))

Y_10 = np.asarray(Y_10)

### Run Second Layer of Supervised NMF

In [ ]:
#model = SSNMF(S_1,6,Y = np.multiply(Y_hot,L),L=L,lam=1000, modelNum=3)
np.random.seed(1)

model_3 = SSNMF(S_1,6,Y = Y_hot,L=L,lam=15, modelNum=3)

N=3000
model_3.mult(numiters = N)
A_1 = model_3.A
S_2 = model_3.S

In [ ]:
B = model.B

Y_pred = np.argmax(np.dot(B,S_2), axis=0)
Y = np.asarray(Y)

print("Accuracy:          {}/{}".format(Y[Y_pred==Y].shape[0], Y.shape[0]))
print("Accuracy known:    {}/{}".format(Y[L[0]==1][Y_pred[L[0]==1]==Y[L[0]==1]].shape[0], Y[L[0]==1].shape[0]))
print("Accuracy unknown:  {}/{}".format(Y[L[0]==0][Y_pred[L[0]==0]==Y[L[0]==0]].shape[0], Y[L[0]==0].shape[0]))

In [ ]:
A1 = A_0
A2 = A_1

keywords = np.empty((12,10), dtype=object)

for i in range(keywords.shape[1]):
    keywords[0,i] = "Topic " + str(i+1)
    keywords[1,i] = "-------"

for i in range(A1.shape[1]):
    col = A1[:,i]
    top = col.argsort()
    top = top[-10:][: :-1]

    keywords[2:,i] = idx_to_word[top] 

print("RANK 10 KEYWORDS:")  
print("------------------")
col_widths = [max([len(keywords[i][j]) for i in range(keywords.shape[0])])+2 for j in range(keywords.shape[1])]
for row in keywords:
    print("".join(row[i].ljust(col_widths[i]) for i in range(0,5)))
print("")
for row in keywords:
    print("".join(row[i].ljust(col_widths[i]) for i in range(5,10)))


keywords = np.empty((12,6), dtype=object)

for i in range(keywords.shape[1]):
    keywords[0,i] = "Topic " + str(i+1)
    keywords[1,i] = "-------"

A = A1 @ A2
for i in range(A.shape[1]):
    col = A[:,i]
    top = col.argsort()
    top = top[-10:][::-1]

    keywords[2:,i] = idx_to_word[top] 

print("RANK 6 KEYWORDS:")  
print("------------------")
col_widths = [max([len(keywords[i][j]) for i in range(keywords.shape[0])])+2 for j in range(keywords.shape[1])]
for row in keywords:
    print("".join(row[i].ljust(col_widths[i]) for i in range(len(row))))

In [ ]:
#calculate reconstruction error
layer1_error = np.linalg.norm(X - A_0 @ S_1)
layer2_error = np.linalg.norm(X - A_0 @ A_1 @ S_2)

print("Layer 1 error...    " + str(layer1_error))
print("Layer 2 error...    " + str(layer2_error))

In [ ]:
A1 = A_0
A2 = A_1

keywords = np.empty((12,10), dtype=object)

for i in range(keywords.shape[1]):
    keywords[0,i] = "Topic " + str(i+1)
    keywords[1,i] = "-------"

for i in range(A1.shape[1]):
    col = A1[:,i]*7/6 -  np.mean(A1, axis=1)
    top = col.argsort()
    top = top[-10:][::-1]

    keywords[2:,i] = idx_to_word[top] 

print("RANK 10 KEYWORDS:")  
print("------------------")
col_widths = [max([len(keywords[i][j]) for i in range(keywords.shape[0])])+2 for j in range(keywords.shape[1])]
for row in keywords:
    print("".join(row[i].ljust(col_widths[i]) for i in range(0,5)))
print("")
for row in keywords:
    print("".join(row[i].ljust(col_widths[i]) for i in range(5,10)))


keywords = np.empty((12,6), dtype=object)

for i in range(keywords.shape[1]):
    keywords[0,i] = "Topic " + str(i+1)
    keywords[1,i] = "-------"

A = A1 @ A2
for i in range(A.shape[1]):
    col = A[:,i]*7/6 - np.mean(A,axis=1)
    top = col.argsort()
    top = top[-10:][::-1]

    keywords[2:,i] = idx_to_word[top] 

print("RANK 6 KEYWORDS:")  
print("------------------")
col_widths = [max([len(keywords[i][j]) for i in range(keywords.shape[0])])+2 for j in range(keywords.shape[1])]
for row in keywords:
    print("".join(row[i].ljust(col_widths[i]) for i in range(len(row))))